In [51]:
import pandas as pd

In [52]:
#### processing each book

## for 0:

def process_zero(df):
    df = df.copy()
    df = df[['handicap', 'odds', 'name']]
    df[['participant_name', 'name']] = df['name'].str.rsplit(' ', 1, expand=True)

    df = df.dropna()
    return df

## for 1: (Pinnacle)

def process_one(df):
    df = df.copy()
    df.dropna()
    return df[['handicap', 'odds', 'participant_name', 'name']]

## for two and six (DraftKings)

def process_two_six(df):
    df = df.copy()
    df = df[['handicap', 'odds', 'name']]

    df[['name', 'participant_name']] = df['name'].str.split(' - ', expand=True)
    df['participant_name'] = df['participant_name'].str.strip()

    df.dropna()

    return df

## for three and five

def process_three_five(df):
    df = df.copy()
    df = df[['handicap', 'odds', 'name', 'description']]

    df[['participant_name', 'description', 'a']] = df['description'].str.rsplit(' ', n=2, expand=True)
    df['participant_name'] = df['participant_name'].str.strip()

    df = df.drop(columns=['description', 'a'])
    df = df.dropna()

    return df


## for seven and eight

def process_seven_eight(df):
    df = df.copy()
    df = df[['handicap', 'odds', 'name']]

    df[['participant_name', 'name', 'a']] = df['name'].str.rsplit(' ', n=2, expand=True)

    df = df.drop(columns=['a'])
    df = df.dropna()

    return df

def process_master(data):
    df = pd.DataFrame.from_dict(data['market']['outcomes'])
    if data['bookie_key'] == "pinnacle":
        df = df.copy()
        df.dropna()
        return df[['handicap', 'odds', 'participant_name', 'name']]
    # elif data['bookie_key'] == "fanduel":
    #     df = df.copy()
    #     df = df[['handicap', 'odds', 'name']]
    #     print(df)
    #     df[['participant_name', 'name']] = df['name'].str.rsplit(' ', 1, expand=True)

    #     df = df.dropna()
    #     return df
    elif data['bookie_key'] == "draftkings" or data['bookie_key'] == "betrivers":
        df = df.copy()
        df = df[['handicap', 'odds', 'name']]

        df[['name', 'participant_name']] = df['name'].str.split(' - ', expand=True)
        df['participant_name'] = df['participant_name'].str.strip()

        df.dropna()

        return df

In [53]:
#### for merging all the books

## agg
def agg(df):    
    result_df = df.groupby(['participant_name', 'name']).agg({'handicap': 'first', 'odds': 'median'}).reset_index()
    return result_df

## pivot
def pivot(df):
    df = df.copy()
    pivot_df = pd.pivot_table(df, values='odds', index=['handicap', 'participant_name'], columns='name').reset_index()
    pivot_df.columns = ['line', 'participant_name', 'over_odds', 'under_odds']

    return pivot_df

## calculate odds
## adapted from Ammar Sulmanjee

def calculate_odds(x, y):
    if x >= 0:
        decimal_odds_1 = 1 + x/100
    else: 
        decimal_odds_1 = 1 + 100/abs(x)
        
    if y >= 0:
        decimal_odds_2 = 1 + y/100
    else: 
        decimal_odds_2 = 1 + 100/abs(y)

    imp_prob1 = (1 / decimal_odds_1) * 100
    imp_prob2 = (1 / decimal_odds_2) * 100

    total_implied_prob = round(imp_prob1 + imp_prob2, 4)
    fair_prob1 = round(imp_prob1 / total_implied_prob * 100, 2)
    fair_prob2 = round(imp_prob2 / total_implied_prob * 100, 2)

    ## 47, 53 == 100
    ## 53 - 50 = 3

    return max(fair_prob1, fair_prob2) - 50

In [64]:
bets = pd.DataFrame(columns = ['participant_name', 'ev', 'market', 'line'])

In [69]:
import os
import json

fantasy_directory = 'NBA_DATA/other/2023-10-24'
bookies_directory = 'NBA_DATA/books/2023-10-24'

for market in os.listdir(fantasy_directory):
    print(market)
    for game in os.listdir(os.path.join(fantasy_directory, market)):
        #load the json file
        with open(os.path.join(fantasy_directory, market, game)) as f:
            data = json.load(f)
            # iterate over the data's "fantasy_books" subfield
            ud = None
            for book in data['fantasy_books']:
                if book['bookie_key'] == "underdog":
                    ud = pd.DataFrame.from_dict(book['market']['lines'])
                    ud = ud[['participant_name', 'line']]
                    ud.columns = [ud.columns[0], 'handicap']
                    

                    # get sportsbook odds
                    with open(os.path.join(bookies_directory, market, game.replace("fantasy", "books"))) as f:
                        bookies_data = json.load(f)
                        books = []
                        for bookie in bookies_data['sportsbooks']:
                            if bookie['bookie_key'] == "pinnacle" or bookie['bookie_key'] == "betrivers" or bookie['bookie_key'] == "draftkings":
                                df = process_master(bookie)
                                books.append(df)
            
                        # we need to concat the books

                        new_datasets = []
                        for dataset in books:
                            dataset = pd.merge(dataset, ud, how='inner', on = ['participant_name', 'handicap'])
                            dataset = agg(dataset)
                            if dataset.shape[0] > 0 and dataset.shape[1] > 0:
                                new_datasets.append(pivot(dataset))
                        
                        concat = pd.concat(new_datasets, axis=0)
                        final = concat.copy()

                        # final = agg(final)

                        print(final)

                        # final = final[final['ev'] >= 4.99]

                        # final['market'] = market

                        # if final.shape[0] > 0: bets = pd.concat([bets, final[['participant_name', 'ev', 'market', 'line']]])



player_threes_over_under
    line    participant_name  over_odds  under_odds
0    1.5       Austin Reaves     -123.0      -105.0
1    2.5  Michael Porter Jr.     -127.0      -105.0
0    0.5       Anthony Davis      140.0      -185.0
1    0.5      Christian Wood     1200.0    -20000.0
2    0.5        Jimmy Butler     -125.0      -105.0
3    1.5      Andrew Wiggins     -130.0       100.0
4    1.5       Austin Reaves      550.0     -1100.0
5    1.5        Bradley Beal      110.0      -140.0
6    1.5        Franz Wagner     -105.0      -125.0
7    1.5   Jaren Jackson Jr.     -110.0      -120.0
8    1.5        Jerami Grant        NaN       105.0
9    2.5       Keegan Murray     -105.0      -120.0
10   2.5  Michael Porter Jr.     1100.0     -6000.0
11   4.5       Stephen Curry     -115.0      -115.0
0    1.5      Andrew Wiggins     -155.0       116.0
1    1.5       Austin Reaves     -134.0       102.0
2    1.5        Bradley Beal     -112.0      -120.0
3    1.5     Cade Cunningham     -107.0